<a href="https://colab.research.google.com/github/lauren-mh16/risesc_public/blob/main/asthma_pm25_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd

In [42]:
df_clarity = pd.read_csv("/content/drive/MyDrive/3. DATASCI CAPSTONE/Data/risesouthcity_april_daily.csv")
df_asthma = pd.read_csv("/content/drive/MyDrive/3. DATASCI CAPSTONE/Data/asthma_clean.csv")
df_census = pd.read_csv("/content/drive/MyDrive/3. DATASCI CAPSTONE/Data/april 23 - datasets/CA_census tracts_demographic_data.csv")
df_avgs = pd.read_csv("/content/pm25_avg_by_site")

In [43]:
df_clarity.loc[df_clarity['datasourceId'] == 'DRYLF3821', 'Name'] = 'Cypress Avenue Home'
df_clarity.loc[df_clarity['datasourceId'] == 'DEVPF7186', 'Name'] = 'Grand Avenue Home'
df_clarity.loc[df_clarity['datasourceId'] == 'DETMG3939', 'Name'] = 'Hawthorne Place Home'

# Distinguish schools
df_clarity.loc[df_clarity['datasourceId'] == 'DTMSK2119', 'Name'] = 'Rollingwood Elementary School'
df_clarity.loc[df_clarity['datasourceId'] == 'DUBTA4581', 'Name'] = 'Portola Elementary School'
df_clarity.loc[df_clarity['datasourceId'] == 'DJTYV8538', 'Name'] = 'Parkside Middle School'

In [44]:
df_clarity_census = pd.read_csv("/content/clarity (1).csv")[['Name', 'Census_Tract', 'Latitude', 'Longitude']].drop_duplicates(subset='Name')
name_updates = {
    'Nora Alvarado Home': 'Hawthorne Place Home',
    'Evelin Pacheco Home': 'Grand Avenue Home',
    'Marita Santos Home': 'Cypress Avenue Home',
    # Schools
    'Parkside Middle': 'Parkside Middle School',
    'Belle Air School': 'Belle Air Elementary School',
    'Rollingwood Elementary': 'Rollingwood Elementary School',
    'Portola Elementary': 'Portola Elementary School',
}

# Apply the updates in df_clarity_census
df_clarity_census['Name'] = df_clarity_census['Name'].replace(name_updates)

df_clarity_census

,Name,Census_Tract,Latitude,Longitude
0,Brentwood Park,6081602400,37.63757,-122.43270
139,Hawthorne Place Home,6081602202,37.65875,-122.41055
290,Grand Avenue Home,6081602201,37.65721,-122.41898
441,Cypress and Pine Playlot,6081602100,37.65876,-122.40700
580,Rise South City Office,6081602202,37.65544,-122.41276
731,San Bruno School District Office,6081604000,37.62292,-122.41611
882,Parkside Middle School,6081603900,37.61580,-122.41844
1033,Buri Buri Park,6081601800,37.65214,-122.44095
1172,Clay Ave Park,6081601603,37.66170,-122.46329
1311,Belle Air Elementary School,6081604200,37.62441,-122.40469


In [45]:
df_asthma_clean = df_asthma.rename(columns={'Location': 'Census_Tract'})[
    ['Census_Tract', 'asthma_rate', 'year', 'Rate Upper Confidence Interval']
]
df_asthma_clean

,Census_Tract,asthma_rate,year,Rate Upper Confidence Interval
0,6081600200,7.4,2014,7.9
1,6081600300,6.9,2014,7.2
2,6081600401,6.7,2014,7.0
3,6081600402,7.1,2014,7.5
4,6081600500,6.6,2014,7.0
...,...,...,...,...
1056,6081613701,9.2,2022,10.4
1057,6081613702,9.5,2022,10.8
1058,6081613800,10.0,2022,11.3
1059,6081613900,9.0,2022,10.2


In [47]:
df_avgs = df_avgs.rename(columns={'2024': 'PM25_2024', '2025': 'PM25_2025'})

In [49]:
# Step 1: Get only the most recent asthma year per Census_Tract
df_asthma_latest = (
    df_asthma_clean.sort_values('year')  # sort to ensure latest is last
    .groupby('Census_Tract', as_index=False)
    .tail(1)
)

# Step 2: Merge in the data
df_merged = (
    df_avgs
    .merge(df_clarity_census, on='Name')
    .merge(df_asthma_latest, on='Census_Tract')
)

,Name,PM25_2024,PM25_2025,Census_Tract,Latitude,Longitude,asthma_rate,year,Rate Upper Confidence Interval
0,Brentwood Park,8.76,5.62,6081602400,37.63757,-122.43270,8.8,2022,9.9
1,Buri Buri Park,8.82,5.73,6081601800,37.65214,-122.44095,8.1,2022,9.2
2,Clay Ave Park,6.74,5.49,6081601603,37.66170,-122.46329,7.4,2022,8.4
3,Cypress Avenue Home,8.88,6.23,6081602202,37.65783,-122.40707,9.1,2022,10.3
4,Cypress and Pine Playlot,11.77,8.59,6081602100,37.65876,-122.40700,9.8,2022,11.1
5,Gardiner Park,8.81,5.81,6081602100,37.66409,-122.40165,9.8,2022,11.1
6,Grand Avenue Home,8.23,5.82,6081602201,37.65721,-122.41898,9.2,2022,10.4
7,Hawthorne Place Home,8.63,6.08,6081602202,37.65875,-122.41055,9.1,2022,10.3
8,Parkside Middle School,7.46,5.02,6081603900,37.61580,-122.41844,8.8,2022,10.0
9,Portola Elementary School,7.61,5.29,6081614000,37.62428,-122.45347,8.1,2022,9.2


In [64]:
df_merged.to_csv('clarity_asthma_merged.csv', index=False)

In [50]:
import folium

# Set the map center (mean of lat/lon)
center_lat = df_merged['Latitude'].mean()
center_lon = df_merged['Longitude'].mean()

# Initialize the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers
for _, row in df_merged.iterrows():
    popup_text = (
        f"<b>{row['Name']}</b><br>"
        f"PM2.5 (2024): {row['PM25_2024']:.2f} µg/m³<br>"
        f"PM2.5 (2025): {row['PM25_2025']:.2f} µg/m³<br>"
        f"Census Tract: {row['Census_Tract']}<br>"
        f"Asthma Rate: {row['asthma_rate']} per 1000 (Year: {int(row['year'])})<br>"
        f"Upper CI: {row['Rate Upper Confidence Interval']}"
    )
    folium.CircleMarker(
        location=(row['Latitude'], row['Longitude']),
        radius=6,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Save and/or display
m.save('pm25_asthma_map.html')

In [52]:
import pandas as pd
import folium
from folium import features

# Set map center
center_lat = df_merged['Latitude'].mean()
center_lon = df_merged['Longitude'].mean()

# Initialize map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Define color by Asthma Rate
def asthma_color(rate):
    if rate < 8:
        return 'green'
    elif 8 <= rate < 9.5:
        return 'yellow'
    else:
        return 'red'

# Define marker shape by site type
def site_type_icon(name):
    name_lower = name.lower()
    if 'school' in name_lower:
        return 'graduation-cap'  # school symbol
    elif 'home' in name_lower:
        return 'home'
    elif 'park' in name_lower or 'playlot' in name_lower:
        return 'tree'
    elif 'office' in name_lower:
        return 'building'
    else:
        return 'map-marker'  # default

# Add markers
for _, row in df_merged.iterrows():
    popup_text = (
        f"<b>{row['Name']}</b><br>"
        f"PM2.5 (2024): {row['PM25_2024']:.2f} µg/m³<br>"
        f"PM2.5 (2025): {row['PM25_2025']:.2f} µg/m³<br>"
        f"Census Tract: {row['Census_Tract']}<br>"
        f"Asthma Rate: {row['asthma_rate']} per 1000 (Year: {int(row['year'])})<br>"
        f"Upper CI: {row['Rate Upper Confidence Interval']}"
    )

    # Set color and icon
    color = asthma_color(row['asthma_rate'])
    icon_type = site_type_icon(row['Name'])

    # Scale marker size based on PM2.5 (2024)
    radius_size = max(6, row['PM25_2024'] * 1.5)  # min size 6

    # Use a Marker with an icon
    folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=color, icon=icon_type, prefix='fa')
    ).add_to(m)

# Add manual legend
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 220px; height: 210px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: white; padding: 10px;">
<b>Legend</b><br>
<b>Asthma Rate Color:</b><br>
<span style="color:green;">■</span> Low (<8 per 1000)<br>
<span style="color:yellow;">■</span> Moderate (8–9.5 per 1000)<br>
<span style="color:red;">■</span> High (>9.5 per 1000)<br><br>
<b>Site Type Icons:</b><br>
🏫 School<br>
🏠 Home<br>
🌳 Park / Playlot<br>
🏢 Office<br>
📍 Other
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save and show
m.save('pm25_asthma_advanced_map.html')
m


<ipython-input-52-d06208ed621b>:57: UserWarning: color argument of Icon should be one of: {'white', 'lightgray', 'beige', 'black', 'darkpurple', 'red', 'orange', 'pink', 'darkblue', 'gray', 'lightgreen', 'purple', 'cadetblue', 'darkred', 'green', 'darkgreen', 'lightred', 'blue', 'lightblue'}.
  icon=folium.Icon(color=color, icon=icon_type, prefix='fa')


In [63]:
import pandas as pd
import folium

# Set map center
center_lat = df_merged['Latitude'].mean()
center_lon = df_merged['Longitude'].mean()

# Initialize map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Define color by PM2.5 (2025) severity
def pm25_2025_color(pm25_2025):
    if pm25_2025 < 6:
        return 'lightblue'
    elif 6 <= pm25_2025 < 9:
        return 'orange'
    else:
        return 'red'

# Define marker shape by site type
def site_type_icon(name):
    name_lower = name.lower()
    if 'school' in name_lower:
        return 'graduation-cap'
    elif 'home' in name_lower:
        return 'home'
    elif 'park' in name_lower or 'playlot' in name_lower:
        return 'tree'
    elif 'office' in name_lower:
        return 'building'
    else:
        return 'map-marker'

# Add markers
for _, row in df_merged.iterrows():
    popup_text = (
        f"<b>{row['Name']}</b><br>"
        f"PM2.5 (2024): {row['PM25_2024']:.2f} µg/m³<br>"
        f"PM2.5 (2025): {row['PM25_2025']:.2f} µg/m³<br>"
        f"Census Tract: {row['Census_Tract']}<br>"
        f"Asthma Rate: {row['asthma_rate']} per 1000 (Year: {int(row['year'])})<br>"
        f"Upper CI: {row['Rate Upper Confidence Interval']}"
    )

    # Set color and icon
    color = pm25_2025_color(row['PM25_2025'])
    icon_type = site_type_icon(row['Name'])

    # Scale marker size based on PM2.5 (2025)
    radius_size = max(6, row['PM25_2025'] * 1.5)  # min size 6

    # Add marker
    folium.Marker(
        location=(row['Latitude'], row['Longitude']),
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color=color, icon=icon_type, prefix='fa')
    ).add_to(m)

# Add a legend (explaining PM2.5 2025 color + site types)
legend_html = """
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 270px; height: 320px;
     border:2px solid grey; z-index:9999; font-size:16px;
     background-color: white; padding: 15px;">
<b>Legend</b><br>
<b>PM2.5 (2025) Color:</br>
<span style="color:lightblue; font-size:18px;">■</span> Low (<6 µg/m³)<br>
<span style="color:orange; font-size:18px;">■</span> Moderate (6–9 µg/m³)<br>
<span style="color:red; font-size:18px;">■</span> High (>9 µg/m³)<br><br>
<b>Site Type Icons:</br>
<span style="font-size:18px;">🎓</span> School<br>
<span style="font-size:18px;">🏠</span> Home<br>
<span style="font-size:18px;">🌳</span> Park / Playlot<br>
<span style="font-size:18px;">🏢</span> Office<br>
<span style="font-size:18px;">📍</span> Other
</div>
"""

m.get_root().html.add_child(folium.Element(legend_html))

# Save and show
m.save('pm25_2025_colormap.html')
m
